    작성일 : 20/03/08
    작성자 : 김은주
    
    작성내용 : 무신사 상품 랭킹 크롤링_최종프로젝트 모델링 반영
    
    테이블생성 : 
    set names utf8;
    CREATE TABLE IF NOT EXISTS FINAL_R_TOP(
    PRDT_CODE         VARCHAR(15),
    BRD_CODE          VARCHAR(50),
    PRDT_NAME         VARCHAR(300),
    PRDT_RANK         VARCHAR(5),
    PRDT_SEX          VARCHAR(2),
    LOAD_DATE       TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
    )default character set utf8;
    
    set names utf8;
    CREATE TABLE IF NOT EXISTS FINAL_R_OUTER(
    PRDT_CODE         VARCHAR(15),
    BRD_CODE          VARCHAR(50),
    PRDT_NAME         VARCHAR(300),
    PRDT_RANK         VARCHAR(5),
    PRDT_SEX          VARCHAR(2),
    LOAD_DATE       TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
    )default character set utf8;
    
    set names utf8;
    CREATE TABLE IF NOT EXISTS FINAL_R_BOTTOM(
    PRDT_CODE         VARCHAR(15),
    BRD_CODE          VARCHAR(50),
    PRDT_NAME         VARCHAR(300),
    PRDT_RANK         VARCHAR(5),
    PRDT_SEX          VARCHAR(2),
    LOAD_DATE       TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
    )default character set utf8;
    
    set names utf8;
    CREATE TABLE IF NOT EXISTS FINAL_R_SNEAKER(
    PRDT_CODE         VARCHAR(15),
    BRD_CODE          VARCHAR(50),
    PRDT_NAME         VARCHAR(300),
    PRDT_RANK         VARCHAR(5),
    PRDT_SEX          VARCHAR(2),
    LOAD_DATE       TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
    )default character set utf8;
    

In [1]:
import requests, pymysql, math, re, time
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [18]:
# ## DB 연결 데이터 
# config = {"host":"192.168.0.12", "user":"musinsa", "passwd":"1111", "db":"PR_TEST"}
# conn = pymysql.connect(**config)

In [2]:
## DB 연결 데이터 
config = {"host":"127.0.0.1", "user":"root", "passwd":"1111", "db":"musinsa_test"}
conn = pymysql.connect(**config)

categorys = {'상의':'001', '아우터':"002", '바지':'003', '가방':"004", '신발':'005', '시계':'006',
            '모자':'007', '레그웨어/속옷':'008', '안경':'009', '액세서리':'011', '디지털/테크':'012',
            '생활/취미/예술':'013', '책/음악/티켓':'014', '코스메틱':'015', '스포츠/용품':'017',
            '스니커':'018', '원피스':'020', '반려동물':'021', '스커트':'022'}
tables = {'001':'FINAL_R_TOP', '002':'FINAL_R_OUTER', '003':'FINAL_R_BOTTOM', '004':'FINAL_R_BAG',
          '005':'FINAL_R_SHOE', '006':'FINAL_R_WATCH', '007':'FINAL_R_HAT', '008':'FINAL_R_UNDER',
          '009':'FINAL_R_GLASS', '011':'FINAL_R_ACC', '012':'FINAL_R_DIGITAL', '013':'FINAL_R_ART',
          '014':'FINAL_R_BOOK', '015':'FINAL_R_COSMETIC', '017':'FINAL_R_SPORT', '018':'FINAL_R_SNEAKER',
          '020':'FINAL_R_ONEPIECE', '021':'FINAL_R_PET', '022':'FINAL_R_SKIRT'}

# html 파싱
def get_html(url):
    html = ""
    resp = requests.get(url)
    
    if resp.status_code == 200 :
        html = resp.text
        
    return html

In [21]:
# 카테고리 랭킹 크롤링
def crawl_musinsa_rank(url_front, url_mid, url_end, category):
    startTime = time.time()
    ranks = []        #랭크 담기 위한 리스트
    brands = []       #브랜드명(한글)을 담기 위한 리스트
    products = []     #상품명, 링크, 코드 담기 위한 리스트
    
    html = get_html(url_front+category+url_mid+"1"+url_end)
    soup = BeautifulSoup(html)
    
    
    # 랭킹페이지 수 100개 제한
    # 카테고리 랭킹 내 상품 수 (총 상품 수 / 80 올림)
    goods_cnt = math.ceil(int(str.strip(soup.find('span', class_="counter box_num_goods").text.replace(',','').replace('상품 수 : ','').replace('개',''))) / 80)
    if goods_cnt > 100:
        rankPage = list(range(1, 101))
    else:
        rankPage = list(range(1, goods_cnt+1))
    
    # 랭크 페이지 반복문
    for page_range in rankPage[:1]:
        html = get_html(url_front+category+url_mid+str(page_range)+url_end)
        soup = BeautifulSoup(html)
        

        temp= [] #랭크 html소스 담기
        for li in soup('li', class_="li_box") :

            #랭크 뽑기
            for rank_p in li('p', class_="n-label") :
                temp = rank_p.get_text().replace("\t","").replace("\n","").strip().split("위")
                ranks.append(["prdt_rank", temp[0]])
            dict_rank = dict(ranks)
            
            
            #브랜드코드 뽑기
            for brand_p in li('p', class_="item_title") :
                for brand_p2 in brand_p('a') :
                    brd_code = brand_p2.get('href').replace("/app/brand/goods_list/","").strip()
                    brands.append(["brd_code", brd_code])
            dict_brand = dict(brands)                      
            
            
            #상품명, 상품링크, 상품코드 뽑기
            for product_p in li('p', class_="list_info") :
                prdt_code = product_p.a['href'].replace("/app/product/detail/", "").replace("/0","")
                products.append(["prdt_name", product_p.a.get_text().strip()])
                products.append(["prdt_code", prdt_code])
                
                ## PRDT_SEX 가져오기
                html = get_html("https://store.musinsa.com/app/product/detail/"+prdt_code)
                soup = BeautifulSoup(html)
                for p in soup.find_all("p", class_="product_article_contents"):            
                    if p.find("span", class_="txt_gender") != None:
                        products.append(["prdt_sex", p.select("span>span")[0].text.strip()])
            dict_product = dict(products)
            
            
            # DB 적재
            cursor = conn.cursor()
            sql = "INSERT INTO " + tables[category]
            data = [dict_product.get("prdt_code", "오류"),
                    dict_brand.get("brd_code", "오류"),
                    dict_product.get("prdt_name", "오류"),
                    dict_rank.get("prdt_rank", '오류'),
                    dict_product.get("prdt_sex", '오류')]

            cursor.execute(sql+"(PRDT_CODE, BRD_CODE, PRDT_NAME, PRDT_RANK, PRDT_SEX) \
                           VALUES(%s, %s, %s, %s, %s)", data)        
            
            conn.commit()     
        print("--- {} seconds ---".format(time.time() - startTime))
        print("{}페이지 완료 : {}".format(page_range, tables[category]))

In [23]:
c = categorys["상의"]

url_front = "https://store.musinsa.com/app/contents/bestranking/?d_cat_cd=&u_cat_cd="
url_mid = "&range=1d&price=&ex_soldout=&sale_goods=&new_product_yn=&list_kind=&page="
url_end = "&display_cnt=80&sex=&popup=&sale_rate=&price1=&price2=&chk_new_product_yn=&chk_sale=&chk_soldout="

# 함수 실행
crawl_musinsa_rank(url_front, url_mid, url_end, c)

conn.close

--- 115.27582263946533 seconds ---
1페이지 완료 : FINAL_R_TOP


<bound method Connection.close of <pymysql.connections.Connection object at 0x000001DDECA05F88>>

In [13]:
# # 상품랭킹 테이블 PRDT_SEX 컬럼에 상품 코드 기준 성별 값 추가

# def crawl_musinsa_rank(category):
#     startTime = time.time()
    
#     # FINAL_R_카테고리 테이블에서 상품코드 가져오기
#     cursor = conn.cursor()
#     sql = "SELECT DISTINCT prdt_code FROM "+tables[category]+";"
#     cursor.execute(sql)
#     codes = [] #상품코드 담기
#     for i in cursor.fetchall() :
#         codes.append(str(i[0]))
        
#     prdt_sex = []
#     for code in codes[]:
#         html = get_html("https://store.musinsa.com/app/product/detail/"+code)
#         soup = BeautifulSoup(html)
#         # 상품 성별 담기
#         for p in soup.find_all("p", class_="product_article_contents"):            
#             if p.find("span", class_="txt_gender") != None:
#                 prdt_sex = p.select("span>span")[0].text.strip()
        
        
#         # DB 적재
#         cursor = conn.cursor()
#         sql = "UPDATE "+tables[category]+" SET PRDT_SEX='"+prdt_sex+"' WHERE PRDT_CODE='"+code+"';"
#         cursor.execute(sql)        
            
#         conn.commit()
#     print("--- {} seconds ---".format(time.time() - startTime))
#     print(tables[category]+"완료")